In [1]:
from sqlalchemy import create_engine, Column, Integer, String, Date, DateTime, Float, BigInteger, Text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
engine = create_engine("sqlite://", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()

/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_64197/1111253954.py:7: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [2]:
class Person(Base):
    __tablename__ = 'PERSON'
    person_id = Column(Integer, primary_key=True, nullable=False)
    gender_concept_id = Column(Integer, nullable=False)
    year_of_birth = Column(Integer, nullable=False)
    month_of_birth = Column(Integer)
    day_of_birth = Column(Integer)
    birth_datetime = Column(DateTime)
    race_concept_id = Column(Integer, nullable=False)
    ethnicity_concept_id = Column(Integer, nullable=False)
    location_id = Column(Integer)
    provider_id = Column(Integer)
    care_site_id = Column(Integer)
    person_source_value = Column(String(50))
    gender_source_value = Column(String(50))
    gender_source_concept_id = Column(Integer)
    race_source_value = Column(String(50))
    race_source_concept_id = Column(Integer)
    ethnicity_source_value = Column(String(50))
    ethnicity_source_concept_id = Column(Integer)

class ObservationPeriod(Base):
    __tablename__ = 'OBSERVATION_PERIOD'
    observation_period_id = Column(Integer, primary_key=True, nullable=False)
    person_id = Column(Integer, nullable=False)
    observation_period_start_date = Column(Date, nullable=False)
    observation_period_end_date = Column(Date, nullable=False)
    period_type_concept_id = Column(Integer, nullable=False)

# Add more class definitions for other tables following the same pattern

# Create the tables in the database
Base.metadata.create_all(engine)


2024-04-15 16:40:46,355 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:40:46,357 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PERSON")
2024-04-15 16:40:46,358 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-15 16:40:46,358 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PERSON")
2024-04-15 16:40:46,359 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-15 16:40:46,359 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("OBSERVATION_PERIOD")
2024-04-15 16:40:46,360 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-15 16:40:46,360 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("OBSERVATION_PERIOD")
2024-04-15 16:40:46,361 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-04-15 16:40:46,362 INFO sqlalchemy.engine.Engine 
CREATE TABLE "PERSON" (
	person_id INTEGER NOT NULL, 
	gender_concept_id INTEGER NOT NULL, 
	year_of_birth INTEGER NOT NULL, 
	month_of_birth INTEGER, 
	day_of_birth INTEGER, 
	birth_datetime DATETIME, 
	race_concept_id INTEGER NOT NU

In [6]:
from sqlalchemy import create_engine, text

# Connection details - replace with your actual details
DATABASE = 'test'  # The name of your MySQL database
USERNAME = 'root'  # Your MySQL database username
PASSWORD = 'TSUtigers123'  # Your MySQL database password
SERVER = 'localhost'  # The hostname or IP address of your MySQL server

# SQLAlchemy connection string for MySQL using mysqlconnector
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'

# Creating the engine for MySQL
engine = create_engine(connection_string, echo=True)

# Path to your SQL file
sql_file_path = './OMOPCDM_sql_server_5.4_ddl.sql'  # Make sure the SQL commands are compatible with MySQL

# Open, read, and prepare the SQL file
with open(sql_file_path, 'r') as sql_file:
    sql_commands = sql_file.read()

    # Since you're working with MySQL, ensure SQL commands in your file are compatible with MySQL
    # Adjust any placeholder replacements as necessary, depending on your SQL file's content

# Execute the SQL commands on MySQL
with engine.begin() as conn:
    # Assuming your SQL commands are separated by semicolons, execute them individually
    # This approach might need adjustments based on the actual content of your SQL file
    for command in sql_commands.split(';'):
        if command.strip():  # Avoid executing empty commands
            conn.execute(text(command))


2024-04-15 16:42:02,983 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-15 16:42:02,983 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:42:02,985 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-15 16:42:02,985 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:42:02,986 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-15 16:42:02,986 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:42:02,987 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:42:02,988 INFO sqlalchemy.engine.Engine --sql server CDM DDL Specification for OMOP Common Data Model 5.4

--HINT DISTRIBUTE ON KEY (person_id)
CREATE TABLE @cdmDatabaseSchema.PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integ

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '--sql server CDM DDL Specification for OMOP Common Data Model 5.4

--HINT DISTRI' at line 1
[SQL: --sql server CDM DDL Specification for OMOP Common Data Model 5.4

--HINT DISTRIBUTE ON KEY (person_id)
CREATE TABLE @cdmDatabaseSchema.PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integer NULL,
			provider_id integer NULL,
			care_site_id integer NULL,
			person_source_value varchar(50) NULL,
			gender_source_value varchar(50) NULL,
			gender_source_concept_id integer NULL,
			race_source_value varchar(50) NULL,
			race_source_con

In [7]:
from sqlalchemy import create_engine, text
import re

# Connection details - replace with your actual details
DATABASE = 'test'
USERNAME = 'root'
PASSWORD = 'TSUtigers123'
SERVER = 'localhost'

# SQLAlchemy connection string for MySQL using mysqlconnector
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'
engine = create_engine(connection_string, echo=True)

# Path to your SQL file
sql_file_path = './OMOPCDM_sql_server_5.4_ddl.sql'

# Read and prepare the SQL file
with open(sql_file_path, 'r') as sql_file:
    sql_commands = sql_file.read()

# Optional: Remove multiline comments
sql_commands = re.sub(r'/\*.*?\*/', '', sql_commands, flags=re.DOTALL)

# Optional: Remove single-line comments
sql_commands = re.sub(r'--.*$', '', sql_commands, flags=re.MULTILINE)

# Replace placeholder with an empty string or appropriate schema name if needed
sql_commands = sql_commands.replace('@cdmDatabaseSchema.', '')

# Execute the SQL commands on MySQL
with engine.begin() as conn:
    for command in sql_commands.split(';'):
        command = command.strip()
        if command:  # Check if command is not empty
            try:
                conn.execute(text(command))
            except Exception as e:
                print(f"An error occurred executing SQL command:\n{command}\nError: {e}")

print("All commands processed.")


2024-04-15 16:45:34,648 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-15 16:45:34,649 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:45:34,651 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-15 16:45:34,652 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:45:34,653 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-15 16:45:34,653 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:45:34,654 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:45:34,655 INFO sqlalchemy.engine.Engine CREATE TABLE PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integer NULL,
			provider_id integer NULL,
			care_site_id integer NULL,
			person_source_value varchar(50) NULL,
			gender_sour

In [3]:
from sqlalchemy import create_engine, text

DATABASE = 'test'
USERNAME = 'root'
PASSWORD = 'TSUtigers123'
SERVER = 'localhost'

# Assuming USERNAME, PASSWORD, SERVER, and DATABASE variables are defined above this line
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'
engine = create_engine(connection_string, echo=True)

# Path to your SQL file
sql_file_path = './OMOPCDM_sql_server_5.4_ddl.sql'

# Initialize an empty list to hold executable lines
executable_lines = []

with open(sql_file_path, 'r') as sql_file:
    for line in sql_file:
        # Remove the placeholder from each line if it exists
        line = line.replace('@cdmDatabaseSchema.', '')

        # Filter out lines that are comments or empty
        if not line.strip().startswith('--') and line.strip():
            executable_lines.append(line)

# Combine filtered and placeholder-removed lines back into commands, split by semicolon
commands = "".join(executable_lines).split(';')

# Execute commands
with engine.begin() as conn:
    for command in commands:
        command = command.strip()  # Avoid executing empty commands
        if command:  # Check if command is not empty
            conn.execute(text(command))


2024-04-15 16:47:55,499 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-15 16:47:55,500 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:47:55,501 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-15 16:47:55,501 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:47:55,502 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-15 16:47:55,502 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:47:55,503 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:47:55,504 INFO sqlalchemy.engine.Engine CREATE TABLE PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integer NULL,
			provider_id integer NULL,
			care_site_id integer NULL,
			person_source_value varchar(50) NULL,
			gender_sour

ProgrammingError: (mysql.connector.errors.ProgrammingError) 1050 (42S01): Table 'person' already exists
[SQL: CREATE TABLE PERSON (
			person_id integer NOT NULL,
			gender_concept_id integer NOT NULL,
			year_of_birth integer NOT NULL,
			month_of_birth integer NULL,
			day_of_birth integer NULL,
			birth_datetime datetime NULL,
			race_concept_id integer NOT NULL,
			ethnicity_concept_id integer NOT NULL,
			location_id integer NULL,
			provider_id integer NULL,
			care_site_id integer NULL,
			person_source_value varchar(50) NULL,
			gender_source_value varchar(50) NULL,
			gender_source_concept_id integer NULL,
			race_source_value varchar(50) NULL,
			race_source_concept_id integer NULL,
			ethnicity_source_value varchar(50) NULL,
			ethnicity_source_concept_id integer NULL )]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [4]:
sql_file_path = './OMOPCDM_sql_server_5.4_primary_keys.sql'

# Read SQL commands from the file and replace the placeholder
with open(sql_file_path, 'r') as file:
    sql_commands = file.read().replace('@cdmDatabaseSchema.', '')

# Split the modified SQL commands into individual commands
# Assuming each command ends with a ';' and possibly a newline for readability
commands = sql_commands.split(';\n')

# Execute each command
with engine.begin() as connection:
    for command in commands:
        command = command.strip()  # Clean whitespace
        if command:  # Check if command is not empty
            connection.execute(text(command))

2024-04-15 16:48:03,522 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:48:03,524 INFO sqlalchemy.engine.Engine ALTER TABLE PERSON ADD CONSTRAINT xpk_PERSON PRIMARY KEY NONCLUSTERED (person_id)
2024-04-15 16:48:03,524 INFO sqlalchemy.engine.Engine [generated in 0.00085s] {}
2024-04-15 16:48:03,538 INFO sqlalchemy.engine.Engine ROLLBACK


ProgrammingError: (mysql.connector.errors.ProgrammingError) 1068 (42000): Multiple primary key defined
[SQL: ALTER TABLE PERSON ADD CONSTRAINT xpk_PERSON PRIMARY KEY NONCLUSTERED (person_id)]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [5]:
from sqlalchemy.engine.reflection import Inspector

inspector = Inspector.from_engine(engine)

# Retrieve and print all table names using the Inspector
table_names = inspector.get_table_names()
print("Tables in the database:")
for table_name in table_names:
    print(table_name)

2024-04-15 16:48:17,243 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:48:17,244 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `test`
2024-04-15 16:48:17,245 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:48:17,273 INFO sqlalchemy.engine.Engine ROLLBACK
Tables in the database:
CARE_SITE
CDM_SOURCE
COHORT
COHORT_DEFINITION
concept
concept_ancestor
CONCEPT_CLASS
concept_relationship
CONCEPT_SYNONYM
condition_era
condition_occurrence
cost
DEATH
device_exposure
DOMAIN
dose_era
drug_era
drug_exposure
drug_strength
EPISODE
EPISODE_EVENT
fact_relationship
LOCATION
measurement
METADATA
note
note_nlp
observation
OBSERVATION_PERIOD
PAYER_PLAN_PERIOD
person
procedure_occurrence
PROVIDER
RELATIONSHIP
source_to_concept_map
specimen
visit_detail
visit_occurrence
VOCABULARY


/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_64197/1350057838.py:3: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


In [6]:
from sqlalchemy import create_engine, text, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy.engine.reflection import Inspector

# Assuming USERNAME, PASSWORD, SERVER, and DATABASE variables are defined above this line
connection_string = f'mysql+mysqlconnector://{USERNAME}:{PASSWORD}@{SERVER}/{DATABASE}'
engine = create_engine(connection_string, echo=True)

# Your existing code to execute SQL commands from a file...

# Now, let's print the DDL for each table
inspector = Inspector.from_engine(engine)

# Reflect the database schema into a new MetaData instance
metadata = MetaData()
metadata.reflect(bind=engine)

print("DDL for tables in the database:")
for table_name in inspector.get_table_names():
    # Using the reflected metadata to get the table object
    table = metadata.tables[table_name]
    # Generating the CREATE TABLE statement for the table
    ddl = CreateTable(table).compile(engine).string
    print(f"\nDDL for {table_name}:\n{ddl}")


2024-04-15 16:48:23,188 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-04-15 16:48:23,189 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:48:23,190 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-04-15 16:48:23,191 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:48:23,191 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-04-15 16:48:23,192 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:48:23,194 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 16:48:23,194 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `test`
2024-04-15 16:48:23,194 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:48:23,209 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CARE_SITE`
2024-04-15 16:48:23,210 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:48:23,213 INFO sqlalchemy.engine.Engine SHOW CREATE TABLE `CDM_SOURCE`
2024-04-15 16:48:23,214 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 16:48:23,217 INFO sqlalchemy.engin

/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_64197/3633835634.py:12: SADeprecationWarning: The from_engine() method on Inspector is deprecated and will be removed in a future release.  Please use the sqlalchemy.inspect() function on an Engine or Connection in order to acquire an Inspector. (deprecated since: 1.4)
  inspector = Inspector.from_engine(engine)


In [41]:
import re

# Path to your SQL file containing CREATE INDEX statements
sql_file_path = './OMOPCDM_sql_server_5.4_indices.sql'

# Read SQL commands from the file and replace the placeholder
with open(sql_file_path, 'r') as file:
    # Read the entire file content
    sql_commands = file.read()
# Remove multiline comments
sql_commands = re.sub(r'/\*.*?\*/', '', sql_commands, flags=re.DOTALL)

# Replace the placeholder
sql_commands = sql_commands.replace('@cdmDatabaseSchema.', '')

# Split the modified SQL commands into individual commands
commands = sql_commands.split(';')

# Filter out empty commands and commands that are commented out
filtered_commands = []
for command in commands:
    # Trim whitespace and remove single-line comments
    stripped_command = command.strip()
    if stripped_command and not stripped_command.startswith('--'):
        filtered_commands.append(stripped_command)

# Execute each filtered command
with engine.begin() as connection:
    for command in filtered_commands:
        try:
            connection.execute(text(command))
        except Exception as e:
            print(f"An error occurred: {e}")
            print(f"While executing: {command}\n")

print("Index creation commands executed successfully.")

2024-04-06 13:35:19,962 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-06 13:35:19,990 INFO sqlalchemy.engine.Engine CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)
2024-04-06 13:35:19,991 INFO sqlalchemy.engine.Engine [generated in 0.02406s] {}
An error occurred: (mysql.connector.errors.ProgrammingError) 1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'CLUSTERED INDEX idx_person_id ON person (person_id ASC)' at line 1
[SQL: CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)]
(Background on this error at: https://sqlalche.me/e/20/f405)
While executing: CREATE CLUSTERED INDEX idx_person_id ON person (person_id ASC)

2024-04-06 13:35:19,998 INFO sqlalchemy.engine.Engine CREATE INDEX idx_gender ON person (gender_concept_id ASC)
2024-04-06 13:35:19,999 INFO sqlalchemy.engine.Engine [generated in 0.00054s] {}
2024-04-06 13:35:20,035 INFO sqlalchemy.engine.En

In [20]:
import pandas as pd
csv_file_path = 'measurement.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'MEASUREMENT'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

table_name = 'measurement'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-15 23:47:59,607 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-15 23:47:59,629 INFO sqlalchemy.engine.Engine DESCRIBE `test`.`MEASUREMENT`
2024-04-15 23:47:59,630 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-15 23:47:59,676 INFO sqlalchemy.engine.Engine INSERT INTO `MEASUREMENT` (measurement_id, person_id, measurement_concept_id, measurement_date, measurement_datetime, measurement_time, measurement_type_concept_id, operator_concept_id, value_as_number, value_as_concept_id, unit_concept_id, range_low, range_high, provider_id, visit_occurrence_id, visit_detail_id, measurement_source_value, measurement_source_concept_id, unit_source_value, value_source_value) VALUES (%(measurement_id)s, %(person_id)s, %(measurement_concept_id)s, %(measurement_date)s, %(measurement_datetime)s, %(measurement_time)s, %(measurement_type_concept_id)s, %(operator_concept_id)s, %(value_as_number)s, %(value_as_concept_id)s, %(unit_concept_id)s, %(range_low)s, %(range_high)s, %(provider_id)s,

/var/folders/hq/659b4zd16vscxjxb3lxdr18c0000gp/T/ipykernel_64197/496905213.py:9: UserWarning: The provided table name 'MEASUREMENT' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  data.to_sql(name=table_name, con=engine, if_exists='append', index=False)


In [66]:
# loading a single csv file to the table
'''
import pandas as pd
csv_file_path = 'cohort/condition_occurrence.csv'  # Update this to the path of your CSV file
data = pd.read_csv(csv_file_path)

table_name = 'condition_occurrence'
# schema = 'cdmDatabaseSchema'

# Load the DataFrame into MySQL
data.to_sql(name=table_name, con=engine, if_exists='append', index=False)

print("Data has been successfully loaded into the database.")

table_name = 'condition_occurrence'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result'''

'\nimport pandas as pd\ncsv_file_path = \'cohort/condition_occurrence.csv\'  # Update this to the path of your CSV file\ndata = pd.read_csv(csv_file_path)\n\ntable_name = \'condition_occurrence\'\n# schema = \'cdmDatabaseSchema\'\n\n# Load the DataFrame into MySQL\ndata.to_sql(name=table_name, con=engine, if_exists=\'append\', index=False)\n\nprint("Data has been successfully loaded into the database.")\n\ntable_name = \'condition_occurrence\'\nquery = text(f"SELECT * FROM {table_name}")\nresult = pd.read_sql(query, con=engine)\nprint("Data retrieved from the database:")\nresult'

In [65]:
## Insert all the data

import pandas as pd
from sqlalchemy import create_engine, exc
import os
csv_files = [f for f in os.listdir(directory_path) if f.endswith('.csv')]

# Loop through the CSV files and load each one into the corresponding MySQL table
for file_name in csv_files:
    table_name = file_name[:-4]  # Remove the '.csv' part to get the table name
    csv_file_path = os.path.join(directory_path, file_name)
    
    try:
        print(f'Loading data from {table_name}')
        # Read the CSV file into a DataFrame
        data = pd.read_csv(csv_file_path)

        # Convert NaT in datetime columns to None (if applicable)
        datetime_columns = data.select_dtypes(include=['datetime']).columns
        data[datetime_columns] = data[datetime_columns].where(data[datetime_columns].notna(), None)

        # Load the DataFrame into MySQL
        data.to_sql(name=table_name, con=engine, if_exists='append', index=False)
        print(f"Data from {file_name} has been successfully loaded into the {table_name} table in the database.")
    
    except exc.IntegrityError as e:
        print(f"IntegrityError: {e}. Skipping duplicate entries for {table_name}.")
    except Exception as e:
        print(f"An error occurred with {table_name}: {e}")

print("All data has been loaded into the database.")


Loading data from drug_exposure
2024-04-16 01:10:42,331 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:10:42,351 INFO sqlalchemy.engine.Engine DESCRIBE `test`.`drug_exposure`
2024-04-16 01:10:42,367 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-04-16 01:10:42,376 INFO sqlalchemy.engine.Engine INSERT INTO drug_exposure (drug_exposure_id, person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_start_datetime, drug_exposure_end_date, drug_exposure_end_datetime, verbatim_end_date, drug_type_concept_id, stop_reason, refills, quantity, days_supply, sig, route_concept_id, lot_number, provider_id, visit_occurrence_id, visit_detail_id, drug_source_value, drug_source_concept_id, route_source_value, dose_unit_source_value) VALUES (%(drug_exposure_id)s, %(person_id)s, %(drug_concept_id)s, %(drug_exposure_start_date)s, %(drug_exposure_start_datetime)s, %(drug_exposure_end_date)s, %(drug_exposure_end_datetime)s, %(verbatim_end_date)s, %(drug_type_concept_id)s, %(stop_reas

IntegrityError: (mysql.connector.errors.IntegrityError) 1062 (23000): Duplicate entry '12000000000156940' for key 'drug_exposure.PRIMARY'
[SQL: INSERT INTO drug_exposure (drug_exposure_id, person_id, drug_concept_id, drug_exposure_start_date, drug_exposure_start_datetime, drug_exposure_end_date, drug_exposure_end_datetime, verbatim_end_date, drug_type_concept_id, stop_reason, refills, quantity, days_supply, sig, route_concept_id, lot_number, provider_id, visit_occurrence_id, visit_detail_id, drug_source_value, drug_source_concept_id, route_source_value, dose_unit_source_value) VALUES (%(drug_exposure_id)s, %(person_id)s, %(drug_concept_id)s, %(drug_exposure_start_date)s, %(drug_exposure_start_datetime)s, %(drug_exposure_end_date)s, %(drug_exposure_end_datetime)s, %(verbatim_end_date)s, %(drug_type_concept_id)s, %(stop_reason)s, %(refills)s, %(quantity)s, %(days_supply)s, %(sig)s, %(route_concept_id)s, %(lot_number)s, %(provider_id)s, %(visit_occurrence_id)s, %(visit_detail_id)s, %(drug

In [56]:
table_name = 'condition_occurrence'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:07:41,470 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:07:41,472 INFO sqlalchemy.engine.Engine SELECT * FROM condition_occurrence
2024-04-16 01:07:41,473 INFO sqlalchemy.engine.Engine [cached since 1681s ago] {}
2024-04-16 01:07:41,543 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,condition_status_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value
0,12000000000000000,1962086,319835,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,I50.21,45543182,89100005
1,12000000000170174,1962086,4043371,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,K52.1,35208271,89100005
2,12000000000177224,1962086,196152,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,K65.8,35208323,89100005
3,12000000000197606,1962086,37110250,2018-12-03,2018-12-03 15:32:00,None,None,44786627,4230359,None,None,12000000067006497,None,I70.245,45548060,89100005
4,12000000000197610,1962086,317577,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,I70.268,45567210,89100005
5,12000000000206700,1962086,197596,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,K52.1,35208271,89100005
6,12000000000206702,1962086,193782,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,N18.6,45548653,89100005
7,12000000000215368,1962086,4000609,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,K31.9,35208246,89100005
8,12000000000218808,1962086,4281826,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,D72.820,45605380,89100005
9,12000000000227666,1962086,434004,2018-12-03,2018-12-03 15:32:00,None,None,44786629,4230359,None,None,12000000067006497,None,E87.70,45576480,89100005


In [57]:
table_name = 'concept'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:07:53,592 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:07:53,609 INFO sqlalchemy.engine.Engine SELECT * FROM concept
2024-04-16 01:07:53,629 INFO sqlalchemy.engine.Engine [generated in 0.03912s] {}
2024-04-16 01:07:53,668 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,9202,Outpatient Visit,Visit,Visit,Visit,S,OP,1970-01-01,2099-12-31,None
1,903096,PMI: Skip,Observation,PPI,Answer,S,PMI_Skip,2017-04-24,2099-12-31,None
2,3041473,Sodium [Moles/volume] in Venous blood,Measurement,LOINC,Lab Test,S,39791-9,2005-03-14,2099-12-31,None
3,4119499,Not for resuscitation,Observation,SNOMED,Clinical Finding,S,304253006,2002-01-31,2099-12-31,None
4,42708201,budesonide 3 MG Delayed Release Oral Capsule,Drug,RxNorm,Clinical Drug,S,1244214,2012-05-07,2099-12-31,None
5,44908056,CONTOUR NEXT STRIPS,Device,NDC,Device,S,193731221,2013-08-05,2099-12-31,None


In [58]:
table_name = 'device_exposure'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:08:09,071 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:08:09,072 INFO sqlalchemy.engine.Engine SELECT * FROM device_exposure
2024-04-16 01:08:09,073 INFO sqlalchemy.engine.Engine [generated in 0.00472s] {}
2024-04-16 01:08:09,117 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,device_exposure_id,person_id,device_concept_id,device_exposure_start_date,device_exposure_start_datetime,device_exposure_end_date,device_exposure_end_datetime,device_type_concept_id,unique_device_id,production_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,device_source_value,device_source_concept_id,unit_concept_id,unit_source_value,unit_source_concept_id
0,33000000000189640,1061241,4224038,2021-04-06,2021-04-06 04:00:00,2021-04-11,2021-04-11 18:39:00,32817,None,None,NaN,None,3.300000e+16,None,336623009.0,4224038.0,None,None,None
1,33000000000240287,1061241,4224038,2021-06-29,2021-06-29 12:25:00,None,NaT,32817,None,None,NaN,None,NaN,None,336623009.0,4224038.0,None,None,None
2,33000000000428231,1061241,4224038,2021-06-29,2021-06-29 13:20:00,None,NaT,32817,None,None,NaN,None,3.300000e+16,None,336623009.0,4224038.0,None,None,None
3,33000000000490287,1061241,4097216,2021-04-05,2021-04-05 15:18:00,2021-04-05,2021-04-04 21:04:00,32817,None,None,NaN,None,3.300000e+16,None,None,NaN,None,None,None
4,52000000000005144,3483034,44844102,2020-04-11,2020-04-10 23:00:00,2021-08-08,2021-08-08 16:16:00,0,None,None,NaN,None,NaN,None,11917013417.0,44844102.0,None,None,None
5,52000000000005145,3483034,44844102,2021-09-18,2021-09-18 00:00:00,None,NaT,0,None,None,100.0,None,NaN,None,11917013417.0,44844102.0,None,None,None
6,52000000000012375,3483034,44908056,2016-07-14,2016-07-14 05:34:21,2016-10-12,2016-10-12 05:34:21,0,None,None,100.0,None,3.300000e+16,None,193731221.0,44908056.0,None,None,None
7,52000000000012376,3483034,44908056,2016-07-14,2016-07-14 19:15:25,2016-10-12,2016-10-12 19:15:25,0,None,None,100.0,None,3.300000e+16,None,193731221.0,44908056.0,None,None,None


In [59]:
table_name = 'drug_exposure'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:08:21,569 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:08:21,573 INFO sqlalchemy.engine.Engine SELECT * FROM drug_exposure
2024-04-16 01:08:21,574 INFO sqlalchemy.engine.Engine [generated in 0.00832s] {}
2024-04-16 01:08:21,606 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,sig,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value
0,12000000000156714,1962086,19072176,2018-11-13,2018-11-12 18:00:00,None,None,None,38000180,None,...,None,4132161,None,None,12000000067006497,None,307675,19072176,26643006,None
1,12000000000156731,1962086,19079524,2018-11-14,2018-11-13 18:00:00,None,None,None,38000180,None,...,None,4171047,None,None,12000000067006497,None,313002,19079524,47625008,None
2,12000000000156742,1962086,19049106,2018-11-16,2018-11-15 18:00:00,None,None,None,38000180,None,...,None,4132161,None,None,12000000067006497,None,198116,19049106,26643006,None
3,12000000000156757,1962086,19127952,2018-11-17,2018-11-16 18:00:00,None,None,None,38000180,None,...,None,4132161,None,None,12000000067006497,None,749206,19127952,26643006,None
4,12000000000156766,1962086,19130823,2018-11-18,2018-11-17 18:00:00,None,None,None,38000180,None,...,None,4243022,None,None,12000000067006497,None,800472,19130823,38239002,None
5,12000000000156786,1962086,1127433,2018-11-19,2018-11-18 18:00:00,None,None,None,38000180,None,...,None,4132161,None,None,12000000067006497,None,313782,1127433,26643006,None
6,12000000000156828,1962086,40167259,2018-11-23,2018-11-22 18:00:00,None,None,None,38000180,None,...,None,4132161,None,None,12000000067006497,None,876193,40167259,26643006,None
7,12000000000156881,1962086,19073526,2018-11-27,2018-11-26 18:00:00,None,None,None,38000180,None,...,None,4171047,None,None,12000000067006497,None,308351,19073526,47625008,None
8,12000000000156940,1962086,42708201,2018-12-01,2018-11-30 18:00:00,None,None,None,38000180,None,...,None,4132161,None,None,12000000067006497,None,1244214,42708201,26643006,None
9,12000000000157016,1962086,1707348,2018-12-03,2018-12-02 18:00:00,None,None,None,38000180,None,...,None,4171047,None,None,12000000067006497,None,311683,1707348,47625008,None


In [60]:
table_name = 'measurement'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:08:30,193 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:08:30,197 INFO sqlalchemy.engine.Engine SELECT * FROM measurement
2024-04-16 01:08:30,201 INFO sqlalchemy.engine.Engine [cached since 5437s ago] {}
2024-04-16 01:08:30,214 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,measurement_id,person_id,measurement_concept_id,measurement_date,measurement_datetime,measurement_time,measurement_type_concept_id,operator_concept_id,value_as_number,value_as_concept_id,...,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,unit_source_concept_id,value_source_value,measurement_event_id,meas_event_field_concept_id
0,9000000001248897,1583645,3027172,2020-09-01,2020-09-01 06:20:00,None,3027172,NaN,20.0,None,...,None,9000000000000000,None,10230-1,3027172,%,None,None,None,None
1,9000000006940564,1249930,3027172,2021-02-09,2021-02-09 07:10:00,None,3027172,NaN,0.0,None,...,None,9000000000000000,None,10230-1,3027172,%,None,None,None,None
2,9000000008087223,1338687,3027172,2017-05-15,2017-05-15 06:57:05,None,3027172,NaN,64.8,None,...,None,9000000006052333,None,10230-1,3027172,%,None,None,None,None
3,9000000008555075,1425778,3027172,2021-07-30,2021-07-30 11:42:00,None,44818702,4172703.0,46.0,None,...,None,9000000154992125,None,10230-1,3027172,%,None,None,None,None
4,9000000009861403,2968472,3027172,2022-04-13,2022-04-13 05:39:00,None,3027172,NaN,66.0,None,...,None,9000000156347989,None,10230-1,3027172,%,None,None,None,None
5,9000000011967592,4006664,3027172,2021-03-16,2021-03-16 07:53:00,None,3027172,NaN,0.0,None,...,None,9000000150386816,None,10230-1,3027172,%,None,None,None,None
6,9000000015336945,1946804,3027172,2019-11-23,2019-11-23 11:46:00,None,3027172,NaN,55.0,None,...,None,9000000120068913,None,10230-1,3027172,%,None,None,None,None
7,9000000016735166,3257211,3027172,2020-08-26,2020-08-26 04:38:00,None,3027172,NaN,71.0,None,...,None,9000000122898385,None,10230-1,3027172,%,None,None,None,None
8,9000000017556558,2304536,3027172,2019-05-15,2019-05-15 09:21:00,None,3027172,NaN,67.0,None,...,None,9000000100840411,None,10230-1,3027172,%,None,None,None,None
9,12000000000691613,8249431,3027172,2018-05-08,2018-05-08 06:43:00,None,44818702,0.0,60.0,None,...,None,12000000049946872,None,10230-1,3027172,%,None,None,None,None


In [61]:
table_name = 'procedure_occurrence'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:08:39,829 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:08:39,858 INFO sqlalchemy.engine.Engine SELECT * FROM procedure_occurrence
2024-04-16 01:08:39,864 INFO sqlalchemy.engine.Engine [generated in 0.03678s] {}
2024-04-16 01:08:39,891 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_end_date,procedure_end_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,modifier_source_value
0,12000000000010823,1962086,2746773,2018-11-17,2018-11-16 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,0DB68ZX,2746773,No matching concept
1,12000000000011294,1962086,2746520,2018-11-17,2018-11-16 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,0DB18ZX,2746520,No matching concept
2,12000000000024589,1962086,2747274,2018-11-17,2018-11-16 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,0DBK8ZX,2747274,No matching concept
3,12000000000026569,1962086,2747017,2018-11-17,2018-11-16 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,0DBB8ZX,2747017,No matching concept
4,12000000000030568,1962086,2732505,2018-11-28,2018-11-27 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,047Q3ZZ,2732505,No matching concept
5,12000000000033870,1962086,4302381,2018-12-03,2018-12-03 15:32:00,None,None,44786629,0,None,None,12000000067006497,None,Z78.1,45581073,No matching concept
6,12000000000033924,1962086,2746803,2018-11-17,2018-11-16 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,0DB98ZX,2746803,No matching concept
7,12000000000037872,1962086,2788126,2018-11-28,2018-11-27 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,B41FYZZ,2788126,No matching concept
8,12000000000043552,1962086,2788717,2018-11-30,2018-11-29 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,30233N1,2788717,No matching concept
9,12000000000058054,1962086,2749338,2018-12-03,2018-12-02 18:00:00,None,None,42865905,0,None,None,12000000067006497,None,0DJW4ZZ,2749338,No matching concept


In [63]:
table_name = 'visit_occurrence'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:09:36,315 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:09:36,316 INFO sqlalchemy.engine.Engine SELECT * FROM visit_occurrence
2024-04-16 01:09:36,316 INFO sqlalchemy.engine.Engine [generated in 0.00252s] {}
2024-04-16 01:09:36,323 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitted_from_concept_id,admitted_from_source_value,discharged_to_concept_id,discharged_to_source_value,preceding_visit_occurrence_id
0,12000000009579745,2608345,9202,2016-11-06,2016-11-05 19:00:00,2016-11-06,2016-11-05 19:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
1,12000000010341014,1666573,9202,2016-03-23,2016-03-22 19:00:00,2016-03-23,2016-03-22 19:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
2,12000000040416418,1914891,9202,2015-03-12,2015-03-11 19:00:00,2015-03-12,2015-03-11 19:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
3,12000000041637604,2802860,9202,2015-12-25,2015-12-24 18:00:00,2015-12-25,2015-12-24 18:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
4,12000000043569168,1074017,9202,2015-11-30,2015-11-29 18:00:00,2015-11-30,2015-11-29 18:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
5,12000000043826633,2564900,9202,2016-07-18,2016-07-17 19:00:00,2016-07-18,2016-07-17 19:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
6,12000000043929548,2489476,9202,2016-08-08,2016-08-07 19:00:00,2016-08-08,2016-08-07 19:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
7,12000000044035255,1191336,9202,2016-07-15,2016-07-14 19:00:00,2016-07-15,2016-07-14 19:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
8,12000000044341626,2673884,9202,2016-08-02,2016-08-01 19:00:00,2016-08-02,2016-08-01 19:00:00,44818518,None,None,21,8717,8940,11,8536,12,None
9,12000000067006497,1962086,262,2018-11-13,2018-11-12 18:00:00,2018-12-03,2018-12-02 18:00:00,44818518,None,None,21,8717,8940,11,44814686,Discharge Disposition-E,None


In [62]:
table_name = 'person'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result

2024-04-16 01:09:18,127 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-04-16 01:09:18,130 INFO sqlalchemy.engine.Engine SELECT * FROM person
2024-04-16 01:09:18,132 INFO sqlalchemy.engine.Engine [generated in 0.00903s] {}
2024-04-16 01:09:18,161 INFO sqlalchemy.engine.Engine ROLLBACK
Data retrieved from the database:


,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id


In [ ]:
table_name = 'x'
query = text(f"SELECT * FROM {table_name}")
result = pd.read_sql(query, con=engine)
print("Data retrieved from the database:")
result